In [1]:
import psycopg2
import pandas as pd

Read in sucide dataset csv

In [2]:
AgeSuicide = pd.read_csv(r"C:\Users\17857\OneDrive\Desktop\Data\Age-standardized suicide rates.csv")

In [3]:
AgeSuicide.head()

,Country,Sex,2016,2015,2010,2000
0,Afghanistan,Both sexes,6.4,6.6,7.4,8.1
1,Afghanistan,Male,10.6,10.9,12.5,14.3
2,Afghanistan,Female,2.1,2.1,2.1,1.7
3,Albania,Both sexes,5.6,5.3,7.7,5.8
4,Albania,Male,7.0,6.7,9.5,8.2


In [4]:
CrudeSuicide = pd.read_csv(r"C:\Users\17857\OneDrive\Desktop\Data\Crude suicide rates.csv")

In [5]:
CrudeSuicide.head()

,Country,Sex,80_above,70to79,60to69,50to59,40to49,30to39,20to29,10to19
0,Afghanistan,Both sexes,42.0,11.0,5.5,5.6,6.6,9.2,10.2,3.1
1,Afghanistan,Male,70.4,20.9,9.8,9.3,10.5,15.1,16.3,4.8
2,Afghanistan,Female,20.1,2.3,1.4,1.6,2.3,2.7,3.5,1.2
3,Albania,Both sexes,16.3,8.3,6.0,7.8,9.1,6.1,6.5,5.0
4,Albania,Male,23.2,11.9,8.1,11.4,13.5,8.8,6.3,3.1


Function for creating postgre database

In [6]:
def create_database():
    #connect to default database
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=****")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    #create sparkify database with UTF8 encoding
    cur.execute("DROP DATABASE IF EXISTS suicide")
    cur.execute("CREATE DATABASE suicide")
    
    #close connection to default database
    conn.close()
    
    #connect to sparkify database
    conn = psycopg2.connect("host=127.0.0.1 dbname=suicide user=postgres password=****")
    cur = conn.cursor()
    
    return cur, conn
    

Use function to create database and then login with dbname suicide on sql shell

In [8]:
cur, conn = create_database()

Create tables for suicide database

In [9]:
age_standardized_table_create = ("CREATE TABLE IF NOT EXISTS age_standardized(\
                                country varchar,\
                                sex varchar,\
                                year_2016 numeric,\
                                year_2015 numeric,\
                                year_2010 numeric,\
                                year_2000 numeric);")

In [10]:
cur.execute(age_standardized_table_create)
conn.commit()

In [11]:
crude_table_create = ("CREATE TABLE IF NOT EXISTS crude(country varchar, sex varchar, age_80above numeric, age_70to79 numeric,\
                     age_60to69 numeric, age_50to59 numeric, age_40to49 numeric, age_30to39 numeric, age_20to29 numeric,\
                     age_10to19 numeric);")

In [12]:
cur.execute(crude_table_create)
conn.commit()

Insert values from csv data into tables

In [14]:
age_standardized_insert = ("INSERT INTO age_standardized(country, sex, year_2016, year_2015, year_2010, year_2000)\
VALUES (%s, %s, %s, %s, %s, %s);")

Loop through each row in dataset and insert data into database

In [16]:
for i, row in AgeSuicide.iterrows():
    cur.execute(age_standardized_insert, list(row))

In [17]:
conn.commit()

Check database in sql shell and data is now in there

In [18]:
crude_insert = ("INSERT INTO crude(country, sex, age_80above, age_70to79, age_60to69, age_50to59,  age_40to49,\
age_30to39, age_20to29, age_10to19) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);")

In [19]:
for i, row in CrudeSuicide.iterrows():
    cur.execute(crude_insert, list(row))

In [20]:
conn.commit()